In [2]:
from flask import Flask, request, jsonify
import openai

In [88]:
from openai import OpenAI
client = OpenAI(api_key = "")
setup  = "reply concisely and must in a single line, mention all the moves given, you will receive all the moves fand the last move in a single line, focus on the last move and explain why the move was taken by the current player"
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": setup}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

White: Bf6, Qe1, Qf3, Qh3, Kd2, Kh1, Ra1, Rg1, Rh1, Rxe1#, 
Black: Bxh3, Kh8, Qh6, Rg8, Rxa1, Rf8. 
The move Rxe1# was taken by White to deliver a checkmate, utilizing the rook to capture Black's king.

In [20]:
import pandas as pd

In [32]:
df = pd.read_csv("games.csv")

In [55]:
df = pd.DataFrame(df["moves"])
df.head()

,moves
0,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...
1,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...
2,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...
3,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...
4,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...


In [56]:
df.shape

(20058, 1)

In [43]:
moves_df = pd.DataFrame(columns = ["moves", "explanations"])

In [57]:
def add_rows(new_df, list_move):
    concat = " ".join(list_move)
    # print(concat)
    new_df = new_df.append({"moves": f"why in chess game, player took the following move - {concat}",
                            "explanations": ""}, ignore_index=True)
    return new_df

In [59]:
for index, row in df.iterrows():
    moves = row["moves"].split(" ")
    
    #progressive moves
    target = [moves[0:i+1] for i, move in enumerate(moves)]
    for progressive_move in target:
        moves_df = add_rows(moves_df, progressive_move)
    if index % 99 == 0 :
        print(f"Row {index} completed")

Row 0 completed
Row 99 completed
Row 198 completed
Row 297 completed
Row 396 completed
Row 495 completed
Row 594 completed


KeyboardInterrupt: 

In [61]:
moves_df.tail

<bound method NDFrame.tail of                                                    moves explanations
0      why in chess game, player took the following m...             
1      why in chess game, player took the following m...             
2      why in chess game, player took the following m...             
3      why in chess game, player took the following m...             
4      why in chess game, player took the following m...             
...                                                  ...          ...
69468  why in chess game, player took the following m...             
69469  why in chess game, player took the following m...             
69470  why in chess game, player took the following m...             
69471  why in chess game, player took the following m...             
69472  why in chess game, player took the following m...             

[69473 rows x 2 columns]>

In [62]:
moves_df.to_csv("moves_df.csv")

In [89]:
df_prompt = pd.read_csv("moves_df.csv")

In [90]:
df_prompt.iloc[0].moves

'why in chess game, player took the following move - d4'

In [91]:
df_prompt["explanation"] = ""

In [92]:
# df_prompt.at[0, "explanation"] = "checking"
# df_prompt

In [93]:
df_prompt

,Unnamed: 0,moves,explanation
0,0,"why in chess game, player took the following m...",
1,1,"why in chess game, player took the following m...",
2,2,"why in chess game, player took the following m...",
3,3,"why in chess game, player took the following m...",
4,4,"why in chess game, player took the following m...",
...,...,...,...
10033,10033,"why in chess game, player took the following m...",
10034,10034,"why in chess game, player took the following m...",
10035,10035,"why in chess game, player took the following m...",
10036,10036,"why in chess game, player took the following m...",


In [98]:
for i in range(100):
    stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": df_prompt.iloc[i].moves}],
    stream=True)
    
    explanation = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            explanation += chunk.choices[0].delta.content
            
    df_prompt.at[i, "explanation"] = explanation

KeyboardInterrupt: 

In [100]:
df_prompt.iloc[50]

Unnamed: 0                                                    50
moves          why in chess game, player took the following m...
explanation    This sequence of moves in a chess game is a co...
Name: 50, dtype: object

In [101]:
df_prompt.to_csv("few_exp.csv")